In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from collections import Counter

# for modeling
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train_and_test2.csv
/kaggle/input/mydata/train.csv
/kaggle/input/mydata/test.csv


Read Data

In [51]:
train = pd.read_csv("/kaggle/input/mydata/train.csv")
test = pd.read_csv("/kaggle/input/mydata/test.csv")

check for null values in the data

In [52]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [53]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

change the sex field into int for eaiser maniuplation and visulization

In [54]:
train['Sex'] = np.where(train['Sex'] == 'female', 1, 0)
test['Sex'] = np.where(test['Sex'] == 'female', 1, 0)

show a sample of the data

In [55]:
train.head(3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [56]:
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q


number of columns and rows in the data

In [57]:
 train.shape

(891, 12)

In [58]:
 test.shape

(418, 11)

check for the outlier values in the dtat using the outlier equation we took in class
drop the outlier values

In [59]:
def detect_outliers(data, n, columns):
    outlier_indices = [] 
    for c in columns: 
        Q1 = np.percentile(data[c], 25)
        Q3 = np.percentile(data[c], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = data[(data[c] < Q1 - outlier_step) | (data[c] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

train_outliers_to_drop = detect_outliers(train, 2, ['Age', 'SibSp', 'Parch', 'Fare'])
test_outliers_to_drop = detect_outliers(test, 2, ['Age', 'SibSp', 'Parch', 'Fare'])
train = train.drop(train_outliers_to_drop, axis = 0).reset_index(drop = True)
test = test.drop(test_outliers_to_drop, axis = 0).reset_index(drop = True)
train_outliers_to_drop

[27, 88, 159, 180, 201, 324, 341, 792, 846, 863]

print the droped outlier value

In [60]:
train_outliers_to_drop

[27, 88, 159, 180, 201, 324, 341, 792, 846, 863]

In [61]:
test_outliers_to_drop

[]

check how many values were droped from the data

In [62]:
train.shape

(881, 12)

In [63]:
 test.shape

(418, 11)

In [64]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            170
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          680
Embarked         2
dtype: int64

In [65]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

fill the age column gaps with the mean of the column

In [66]:
train['Age'] = train['Age'].replace(np.nan, train['Age'].mean())
test['Age'] = test['Age'].replace(np.nan, test['Age'].mean())

In [67]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [68]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          680
Embarked         2
dtype: int64

drop the cabin column because the percentage of the missing data is so big

In [69]:
train = train.dropna(axis = 1)

In [70]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
dtype: int64

In [71]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

fill the Fare gap with the median of the column

In [72]:
test['Fare'] = test['Fare'].replace(np.nan, test['Fare'].median())

In [73]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [74]:
train=train.dropna(axis=1, thresh= 0.7 * len(train))
test=test.dropna(axis=1, thresh=0.7 * len(test))

In [75]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
dtype: int64

In [76]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

get the correlation 

In [77]:
corr = train.corr()
corr

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.003089,-0.042702,-0.046007,0.030955,-0.072042,0.004052,0.031211
Survived,-0.003089,1.000000,-0.334097,0.546015,-0.069634,0.003330,0.092819,0.264613
Pclass,-0.042702,-0.334097,1.000000,-0.131245,-0.336822,0.065031,0.016440,-0.555558
Sex,-0.046007,0.546015,-0.131245,1.000000,-0.084051,0.129308,0.245750,0.179277
Age,0.030955,-0.069634,-0.336822,-0.084051,1.000000,-0.290426,-0.179564,0.104997
SibSp,-0.072042,0.003330,0.065031,0.129308,-0.290426,1.000000,0.379281,0.110072
Parch,0.004052,0.092819,0.016440,0.245750,-0.179564,0.379281,1.000000,0.183801
Fare,0.031211,0.264613,-0.555558,0.179277,0.104997,0.110072,0.183801,1.000000


In [78]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
dtype: int64

In [79]:
x_train, x_val, y_train, y_val = train_test_split(train[['Sex', 'Pclass', 'Fare', 'Age']], train.Survived, random_state = 0)
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf.fit(x_train,y_train)
val_rf_results = rf.predict(x_val)
print(mean_absolute_error(y_val, val_rf_results))

0.167420814479638


In [80]:
svc = SVC( random_state=1)
svc.fit(x_train,y_train)
val_svc_results = svc.predict(x_val)
print(mean_absolute_error(y_val, val_svc_results))

0.2986425339366516


In [81]:
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)
val_knn_results = knn.predict(x_val)
print(mean_absolute_error(y_val, val_knn_results))

0.2850678733031674


In [82]:
dt = tree.DecisionTreeClassifier(random_state=1)
dt.fit(x_train,y_train)
val_dt_results = dt.predict(x_val)
print(mean_absolute_error(y_val, val_dt_results))

0.19909502262443438


In [83]:
lr = LogisticRegression(max_iter = 2000)
lr.fit(x_train,y_train)
val_lr_results = lr.predict(x_val)
print(mean_absolute_error(y_val, val_lr_results))

0.18552036199095023


In [84]:
gnb = GaussianNB()
gnb.fit(x_train,y_train)
val_gnb_results = gnb.predict(x_val)
print(mean_absolute_error(y_val, val_gnb_results))

0.22624434389140272


In [85]:
classifiers = []
classifiers.append(LogisticRegression(max_iter = 2000))
classifiers.append(KNeighborsClassifier())
classifiers.append(GaussianNB())
classifiers.append(SVC())
classifiers.append(tree.DecisionTreeClassifier())
classifiers.append(RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1))


len(classifiers)

6

In [86]:
cv_mean = []
cv_std = []
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [87]:
cv_results = []
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier, x_train, y_train, scoring = 'accuracy', cv = 6))

In [88]:
cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 'Cross Validation Std': cv_std, 'Algorithm': ['Naive Bayes', 'KNN', 'Logistic Regression', 
                                 'Random Forest', 'Decission tree','SVC' ]})
cv_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True)

,Cross Validation Mean,Cross Validation Std,Algorithm
0,0.810606,0.045882,SVC
1,0.780303,0.033368,Logistic Regression
2,0.777273,0.050000,Naive Bayes
3,0.748485,0.032213,Decission tree
4,0.684848,0.018680,KNN
5,0.663636,0.013887,Random Forest


In [89]:
rf.fit(train[['Sex', 'Pclass', 'Fare', 'Age']], train.Survived)

RandomForestClassifier(max_depth=5, random_state=1)

In [90]:
predictions = rf.predict(test[['Sex', 'Pclass', 'Fare', 'Age']])

In [91]:
submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
submission.to_csv('submission.csv', index=False)